In [ ]:
'''created a Hive Table'''
''''create external table partd
(
npi string,
nppes_provider_last_org_name string,
nppes_provider_first_name string,
nppes_provider_city string,
nppes_provider_state string,
specialty_description string,
description_flag string,
drug_name string,
generic_name string,
bene_count string,
total_claim_count int,
total_day_supply int,
total_drug_cost int,
bene_count_ge65 string,
bene_count_ge65_suppress_flag string,
total_claim_count_ge65 int,
ge65_suppress_flag string,
total_day_supply_ge65 int,
total_drug_cost_ge65 int,
total_30_day_fill_count int,
total_30_day_fill_count_ge65 int
)
row format serde 'org.apache.hadoop.hive.serde2.OpenCSVSerde'
with serdeproperties(
"seperatorChar"=",",
"quotechar"="\""
)
location '/sravan/partd/'
tblproperties("skip.header.line.count"="1");
*/''''

In [ ]:
from pyspark.sql import Row
head_p=sc.textFile("/sravan/partd/part_d_prescriber_2014.csv").first()
prod=sc.textFile("/sravan/partd/part_d_prescriber_2014.csv").filter(lambda x:x!=head_p).map(lambda x:x.split(",")).map(lambda x:Row(npi=(x[0]+"-"+x[1]+"-"+x[2]+"-"+x[4]),specialty_description=(x[5]),npi_state=(x[4]),product_name=(x[7]),Total_Amount=(x[12]), Total_Quant =(x[10])))
prod.toDF().registerTempTable("partd");
s=sqlContext.sql("select product_name, sum(cast (case when Total_Amount='NA' then 0 else Total_Amount end  as float ))/sum(cast (case when Total_Quant='NA' then 0 else Total_Quant end  as int ))as p from partd group by product_name order by p desc")
sd=s.rdd
sd.map(lambda x:x[0]+","+str(x[1])).repartition(1).saveAsTextFile("/sravan/TopDrugPrices/")

s=sqlContext.sql("select npi, count(distinct product_name) as count_of_products ,sum(Total_Amount) as Total_Amount from partd group by npi order by count_of_products desc")
sd=s.rdd
sd.map(lambda x:x[0]+","+str(x[1])+","+str(x[2])).repartition(1).saveAsTextFile("/sravan/NPIDrugsAmount2/")

s=sqlContext.sql("select specialty_description, sum(Total_Quant) as Total_Quant,sum(Total_Amount) as Total_Amount from  partd group by specialty_description order by Total_Amount desc")
sd=s.rdd
sd.map(lambda x:x[0]+","+str(x[1])+","+str(x[2])).repartition(1).saveAsTextFile("/sravan/byspecialty1/")


In [1]:
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import plotly.plotly as py
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import pandas as pd

In [2]:
npidrugs=pd.read_csv("Documents\cloudera-quickstart-vm-5.13.0-0-virtualbox\part-00000-npidrugs_name",names=['NPI', 'CountOfDrugsPrescribed','TotalAmount'], dtype={'NPI': object},header=None)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning:

Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.



In [11]:
npidrugsf=npidrugs.iloc[0:100,]
init_notebook_mode(connected=True)


In [12]:
trace1 = go.Bar(
    x=npidrugsf.NPI,
    y=npidrugsf.CountOfDrugsPrescribed,
    name='DistinctDrugsPrescribed'
)
trace2 = go.Bar(
    x=npidrugsf.NPI,
    y=npidrugsf.TotalAmount/100000,
    name='Total$AmountOfDrugsPrescribedIn100Thousands'
)


data = [trace1,trace2]
layout = go.Layout(
    barmode='group'
    , margin=dict(b=200)
    
)

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='DrugAmount_DrugQuantity_By_Doctor.html')

In [13]:
byspecialty=pd.read_csv("Documents\cloudera-quickstart-vm-5.13.0-0-virtualbox\part-00000-byspecialty",names=['Specialty', 'TotalQuantity','TotalAmount',],header=None)

In [14]:
trace1 = go.Bar(
    x=byspecialty.Specialty,
    y=byspecialty.TotalAmount,
    name='AmountBySpecialty'
)
trace2 = go.Bar(
    x=byspecialty.Specialty,
    y=byspecialty.TotalQuantity ,
    name='ClaimsbySpecialty'
)


data = [trace1,trace2]
layout = go.Layout(
    barmode='group'
    , margin=dict(b=200)
)

fig = go.Figure(data=data,layout=layout)
iplot(fig, filename='DrugAmount_DrugQuantity_By_Specialty.html')

In [15]:
Topdrugs=pd.read_csv("Documents\cloudera-quickstart-vm-5.13.0-0-virtualbox\part-00000-TopDrugs",names=['DrugName','DrugCost']
,header=None)

In [16]:
trace1 = go.Bar(
    x=Topdrugs.DrugName,
    y=Topdrugs.DrugCost,
    name='DrugPrice'
)


data = [trace1]
layout = go.Layout(
    barmode='group'
    , margin=dict(b=200)
)

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='PricesOfDrugs.html')